# Procesamiento de Elecciones de Gobernador - Manzanillo (2016-2024)
## Objetivo
El propósito de este notebook es limpiar, estandarizar y estructurar los resultados electorales para las elecciones de Gobernador en Manzanillo, abarcando las elecciones de 2016 y 2021. El objetivo final es generar archivos de datos limpios y consistentes que puedan ser fácilmente consolidados en una única base de datos para su posterior análisis y visualización geoespacial.

### Manzanillo Gobernador 2016

In [7]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Gobernador')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Gobernador')

input_file = 'Manzanillo_Gobernador_2016.csv'
output_file = 'Manzanillo_Gobernador_2016_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ESTADO              410 non-null    int64 
 1   NOMBRE_ESTADO       410 non-null    object
 2   DISTRITO            410 non-null    int64 
 3   CABECERA_DISTRITAL  410 non-null    object
 4   seccion             410 non-null    int64 
 5   ID_CASILLA          410 non-null    int64 
 6   casilla             410 non-null    object
 7   EXT_CONTIGUA        410 non-null    int64 
 8   UBICACION_CASILLA   410 non-null    object
 9   TIPO_ACTA           410 non-null    object
 10  pan                 410 non-null    object
 11  pri                 410 non-null    object
 12  prd                 410 non-null    object
 13  pvem                410 non-null    object
 14  pt                  410 non-null    object
 15  mc                  410 non-null    object
 16  na      

,ESTADO,NOMBRE_ESTADO,DISTRITO,CABECERA_DISTRITAL,seccion,ID_CASILLA,casilla,EXT_CONTIGUA,UBICACION_CASILLA,TIPO_ACTA,...,pri_pvem,pri_pt,pri_na,pvem_pt,pvem_na,pt_na,voto_no_registrado,voto_nulo,voto_total,lista_nominal
0,6,COLIMA,2,MANZANILLO,0,1,B·sica,0,No urbana,GOB,...,0,0,0,0,0,0,0,0,7,33
1,6,COLIMA,2,MANZANILLO,170,1,B·sica,0,No urbana,GOB,...,2,0,0,0,0,0,0,5,331,580
2,6,COLIMA,2,MANZANILLO,170,1,Contigua,0,No urbana,GOB,...,1,0,0,0,0,0,0,9,359,579
3,6,COLIMA,2,MANZANILLO,170,2,Contigua,0,No urbana,GOB,...,2,0,0,0,0,0,0,7,320,579
4,6,COLIMA,2,MANZANILLO,171,1,B·sica,0,Urbana,GOB,...,1,0,1,1,0,0,0,5,307,506


In [8]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Gobernador')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Gobernador')

input_file = 'Manzanillo_Gobernador_2016.csv'
output_file = 'Manzanillo_Gobernador_2016_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path, encoding='latin-1')
print("Datos originales de 2016 (versión definitiva) cargados.")

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto innecesarias
columnas_a_eliminar = [
    'estado', 'nombre_estado', 'distrito', 'cabecera_distrital', 
    'id_casilla', 'ext_contigua', 'ubicacion_casilla', 'tipo_acta'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) PASO CLAVE: CORREGIR TIPOS DE DATO
columnas_a_convertir = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pri_pvem_na_pt', 'pri_pvem_pt', 'pri_pvem_na', 'pri_pt_na',
    'pvem_pt_na', 'pri_pvem', 'pri_pt', 'pri_na', 'pvem_pt',
    'pvem_na', 'pt_na', 'voto_no_registrado', 'voto_nulo'
]
# Chequeo de posibles columnas duplicadas que pandas renombra
if 'pvem_pt.1' in df_mapped.columns:
    columnas_a_convertir.append('pvem_pt.1')

for col in columnas_a_convertir:
    if col in df_mapped.columns:
        df_mapped[col] = pd.to_numeric(df_mapped[col], errors='coerce')

df_mapped = df_mapped.fillna(0)

# d) Manejar columnas duplicadas
if 'pvem_pt.1' in df_mapped.columns:
    print("Combinando columna duplicada 'pvem_pt.1'")
    df_mapped['pvem_pt'] = df_mapped['pvem_pt'] + df_mapped['pvem_pt.1']
    df_mapped = df_mapped.drop(columns=['pvem_pt.1'])

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2016
competidores_2016 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pri_pvem_na_pt', 'pri_pvem_pt', 'pri_pvem_na', 'pri_pt_na',
    'pvem_pt_na', 'pri_pvem', 'pri_pt', 'pri_na', 'pvem_pt',
    'pvem_na', 'pt_na'
]
partidos_morena_2016 = ['morena']
oposicion_2016 = [col for col in competidores_2016 if col != 'morena']

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2016].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2016].sum(axis=1)

# c) Composición del Voto "Morena"
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = 0

# d) Ganador por Casilla
df_competidores = df_mapped[competidores_2016]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']


# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'es',
    'pri_pvem_na_pt', 'pri_pvem_pt', 'pri_pvem_na', 'pri_pt_na',
    'pvem_pt_na', 'pri_pvem', 'pri_pt', 'pri_na', 'pvem_pt',
    'pvem_na', 'pt_na',
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2016 (versión definitiva) con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2016 guardado en: {output_path}")

Datos originales de 2016 (versión definitiva) cargados.

Datos limpios de 2016 (versión definitiva) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  410 non-null    int64  
 1   casilla                  410 non-null    object 
 2   lista_nominal            410 non-null    int64  
 3   pan                      410 non-null    float64
 4   pri                      410 non-null    float64
 5   prd                      410 non-null    float64
 6   pvem                     410 non-null    float64
 7   pt                       410 non-null    float64
 8   mc                       410 non-null    float64
 9   na                       410 non-null    float64
 10  morena                   410 non-null    float64
 11  es                       410 non-null    float64
 12

### Manzanillo Gobernador 2021

In [10]:
input_file = "Manzanillo_Gobernador_2021.csv"
output_file = 'Manzanillo_Gobernador_2021_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CIRCUNSCRIPCION           0 non-null      float64
 1   ID_ESTADO                 239 non-null    int64  
 2   NOMBRE_ESTADO             239 non-null    object 
 3   ID_DISTRITO_LOCAL         239 non-null    int64  
 4   CABECERA_DISTRITAL_LOCAL  239 non-null    object 
 5   ID_MUNICIPIO              239 non-null    int64  
 6   MUNICIPIO                 239 non-null    object 
 7   seccion                   239 non-null    int64  
 8   casilla                   239 non-null    object 
 9   ID_CASILLA                239 non-null    int64  
 10  EXT_CONTIGUA              239 non-null    int64  
 11  pan                       239 non-null    int64  
 12  pri                       239 non-null    int64  
 13  prd                       239 non-null    int64

In [11]:
df.head(5)

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO,MUNICIPIO,seccion,casilla,ID_CASILLA,...,pri_prd,votos_validos,votos_no_registrados,voto_nulo,voto_total,lista_nominal,ESTATUS_ACTAS,TRIBUNAL,OBSERVACIONES,RUTA_ACTA
0,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,200,B,1,...,0,340,7,5,352,709,GRUPO DE RECUENTO,NaN,NaN,NaN
1,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,B,1,...,0,284,2,7,293,605,ACTA DEL CONSEJO,NaN,NaN,NaN
2,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,1,...,0,417,0,5,422,605,GRUPO DE RECUENTO,NaN,NaN,NaN
3,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,2,...,0,293,1,7,301,604,ACTA CASILLA,NaN,NaN,NaN
4,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,3,...,0,295,0,6,301,604,ACTA DEL CONSEJO,NaN,NaN,NaN


In [12]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columna a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas de contexto innecesarias
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito_local', 
    'cabecera_distrital_local', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'votos_validos', 'estatus_actas', 'tribunal', 
    'observaciones', 'ruta_acta'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# NO SE NECESITA: Conversión de tipos de dato, ya son numéricos.

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para Gobernador 2021
competidores_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena_na', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]
partidos_morena_2021_total = ['morena_na', 'pt', 'pvem'] # Alianza de facto por la gubernatura
oposicion_2021 = [
    'pan', 'pri', 'prd', 'mc', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2021_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2021].sum(axis=1)

# c) Composición del Voto "Morena" (Análisis Fino)
df_mapped['voto_morena_solo'] = 0 # No había opción de votar por Morena solo
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2021_total].sum(axis=1)

# d) Ganador por Casilla (Análisis Micro)
df_competidores = df_mapped[competidores_2021]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']


# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Columnas de votos originales
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena_na', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    # Analíticas
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    # Cierre - renombrando para consistencia
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
# Renombrar 'votos_no_registrados' a 'voto_no_registrado' por si acaso
if 'votos_no_registrados' in df_mapped.columns:
    df_mapped = df_mapped.rename(columns={'votos_no_registrados': 'voto_no_registrado'})

df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2021 (Gobernador) con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2021 (Gobernador) guardado en: {output_path}")



Datos limpios de 2021 (Gobernador) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  239 non-null    int64 
 1   casilla                  239 non-null    object
 2   lista_nominal            239 non-null    int64 
 3   pan                      239 non-null    int64 
 4   pri                      239 non-null    int64 
 5   prd                      239 non-null    int64 
 6   pvem                     239 non-null    int64 
 7   pt                       239 non-null    int64 
 8   mc                       239 non-null    int64 
 9   morena_na                239 non-null    int64 
 10  rsp                      239 non-null    int64 
 11  fxm                      239 non-null    int64 
 12  pan_pri_prd              239 non-null    int64 
 13  pan_pri                

## Base consolidada elecciones Gobernador en Manzanillo 2016-2021

In [14]:
import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'
# CAMBIO: Apuntamos a la carpeta de Gobernador
input_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Gobernador')
output_folder = os.path.join(BASE_DIR, '03_base_consolidada')
output_file = 'gobernador_consolidado_2016_2021.csv'

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS LIMPIOS ---
search_pattern = os.path.join(input_folder, '*_limpio.csv')
clean_files = sorted(glob.glob(search_pattern))

if not clean_files:
    print("No se encontraron archivos limpios de Gobernador para consolidar.")
else:
    all_dfs = []
    print(f"Se encontraron {len(clean_files)} archivos para consolidar:")
    for file_path in clean_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")
        
        year = int(os.path.basename(file_path).split('_')[-2])
        df = pd.read_csv(file_path)
        df['anio'] = year
        # Añadimos columna para identificar el tipo de elección
        df['tipo_eleccion'] = 'Gobernador' 
        
        all_dfs.append(df)

   # --- 3. CONSOLIDAR, ORDENAR Y GUARDAR ---
    
# Unimos todos los dataframes de la lista
consolidated_df = pd.concat(all_dfs, ignore_index=True, sort=False)
    
# -- INICIA BLOQUE DE ORDENAMIENTO --

# Lista con el orden de columnas sugerido para GOBERNADOR
orden_gobernador = [
    # 1. Identificadores y Contexto
    'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
    
    # 2. Métricas de Competencia (Análisis Micro)
    'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
    'segundo_lugar_votos', 'margen_victoria',
    
    # 3. Métricas de Bloques (Análisis Macro)
    'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados',
    
    # 4. Votos Crudos (Partidos Principales)
    'morena', 'pan', 'pri', 'prd', 'pvem', 'pt', 'mc',
    
    # 5. Votos Crudos (Otros Partidos y Coaliciones de Gobernador)
    'es', 'fxm', 'na', 'morena_na', 'pan_pri', 'pan_pri_prd', 'pan_prd', 
    'pri_na', 'pri_pt', 'pri_pt_na', 'pri_pvem', 'pri_pvem_na', 'pri_pvem_na_pt',
    'pri_pvem_pt', 'pri_prd', 'pt_na', 'pvem_na', 'pvem_pt', 'pvem_pt_na', 'rsp',
    
    # 6. Votos de Cierre
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]

# Para evitar errores, nos aseguramos de ordenar solo con las columnas que existen en el DF
columnas_existentes = [col for col in orden_gobernador if col in consolidated_df.columns]
consolidated_df_ordenado = consolidated_df[columnas_existentes]

# -- FINALIZA BLOQUE DE ORDENAMIENTO --

# Rellenamos los valores nulos con 0
consolidated_df_final = consolidated_df_ordenado.fillna(0)
    
# Asegurarse de que el directorio de salida exista
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)

# Guardar la base de datos FINAL (ordenada y sin nulos)
consolidated_df_final.to_csv(output_path, index=False)
    
print("\n¡Base de datos de Gobernador consolidada y ordenada creada exitosamente!")
print(f"Guardada en: {output_path}")
print("\nResumen de la base consolidada final:")
# Imprimir el info del dataframe FINAL
consolidated_df_final.info()

Se encontraron 2 archivos para consolidar:
  -> Cargando: Manzanillo_Gobernador_2016_limpio.csv
  -> Cargando: Manzanillo_Gobernador_2021_limpio.csv

¡Base de datos de Gobernador consolidada y ordenada creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/03_base_consolidada/gobernador_consolidado_2016_2021.csv

Resumen de la base consolidada final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  649 non-null    int64  
 1   casilla                  649 non-null    object 
 2   anio                     649 non-null    int64  
 3   tipo_eleccion            649 non-null    object 
 4   lista_nominal            649 non-null    int64  
 5   ganador_casilla_partido  649 non-null    object 
 6   ganador_casilla_votos    649 non-null    float64
 7   segundo_lugar_partido    649 non-null    ob